In [1]:
import tensorflow as tf
# tf.get_logger().setLevel('INFO')
from functools import partial
import matplotlib.pyplot as plt
import IPython.display as display
import numpy as np
# import tensorflow_addons as tfa

2022-04-17 12:11:57.750801: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-17 12:11:57.750849: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/usr/lib/python3/dist-packages/Crypto/Random/Fortuna/FortunaGenerator.py:28: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if sys.version_info[0] is 2 and  sys.version_info[1] is 1:
/usr/lib/python3/dist-packages/Crypto/Random/Fortuna/FortunaGenerator.py:28: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if sys.version_info[0] is 2 and  sys.version_info[1] is 1:


In [2]:
strategy = tf.distribute.get_strategy()
print("Number of replicas:", strategy.num_replicas_in_sync)

Number of replicas: 1


In [6]:
# from google.colab import drive
# drive.mount('/content/drive')

# Load the data

In [5]:

AUTOTUNE = tf.data.AUTOTUNE
BATCH_SIZE = 32
# IMAGE_SIZE = [1024, 1024]

In [23]:


FILENAMES_PATH = "American Sign Language Letters.v1-v1.tfrecord/"

TRAINING_FILENAMES =  FILENAMES_PATH + "train/Letters.tfrecords"
VALID_FILENAMES = FILENAMES_PATH + "valid/Letters.tfrecords"
TEST_FILENAMES = FILENAMES_PATH + "test/Letters.tfrecords"

print("Train TFRecord Files:", len(TRAINING_FILENAMES))
print("Validation TFRecord Files:", len(VALID_FILENAMES))
print("Test TFRecord Files:", len(TEST_FILENAMES))

Train TFRecord Files: 69
Validation TFRecord Files: 69
Test TFRecord Files: 68


In [24]:
# example = TRAINING_FILENAMES
labeled = FILENAMES_PATH + "/train/Letters_label_map.pbtxt"
# print(example)

In [31]:
# Create the dataset object for tfrecord file(s)

def load_dataset(tf_filenames):
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = False  # disable order, increase speed
    dataset = tf.data.TFRecordDataset(tf_filenames) 
    
    dataset = dataset.with_options(ignore_order)  
    
    return dataset

In [32]:
# Decoding function
def parse_record(record):
    tfrecord_feat_format = (
                {
                    "image/encoded": tf.io.FixedLenFeature([], tf.string),
                    "image/filename": tf.io.FixedLenFeature([], tf.string),
                    "image/format": tf.io.FixedLenFeature([], tf.string),
                    "image/height": tf.io.FixedLenFeature([], tf.int64),
                    "image/object/bbox/xmax": tf.io.FixedLenFeature([], tf.float32),
                    "image/object/bbox/xmin": tf.io.FixedLenFeature([], tf.float32),
                    "image/object/bbox/ymax": tf.io.FixedLenFeature([], tf.float32),
                    "image/object/bbox/ymin": tf.io.FixedLenFeature([], tf.float32),
                    "image/object/class/label": tf.io.FixedLenFeature([], tf.int64),
                    "image/object/class/text": tf.io.FixedLenFeature([], tf.string),
                    "image/width": tf.io.FixedLenFeature([], tf.int64),
                }
            )
    
    
    
    example = tf.io.parse_single_example(record, tfrecord_feat_format)



    IMAGE_SIZE = [400, 400]

    image =  tf.io.decode_jpeg(example["image/encoded"], channels=3)
    image = tf.cast(image, tf.float32)

    xmax = tf.cast(example["image/object/bbox/xmax"], tf.int32)
    xmin = tf.cast(example["image/object/bbox/xmin"], tf.int32)
    ymax = tf.cast(example["image/object/bbox/ymax"], tf.int32)
    ymin = tf.cast(example["image/object/bbox/ymin"], tf.int32)

    box_width = xmax - xmin
    box_height = ymax - ymin
    image = tf.image.crop_to_bounding_box(image, ymin, xmin, box_height, box_width)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, IMAGE_SIZE)

    image = tf.image.random_flip_left_right(image)
    # image = tfa.image.rotate(image, 40, interpolation='NEAREST')


    # image = tf.cast(image, "uint8")
    # image = tf.image.encode_jpeg(image, format='rgb', quality=100)
    

    label = example["image/object/class/label"]
    label = tf.cast(label, tf.int32)
    # label = tf.one_hot(label, depth=26)

    
    return (image, label)

In [33]:
def get_dataset(filenames):
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = False  # disable order, increase speed
    dataset = tf.data.TFRecordDataset(filenames) 
    
    dataset = dataset.with_options(ignore_order) 
    
    dataset = dataset.map(parse_record, num_parallel_calls=AUTOTUNE)
    dataset = dataset.cache()
    
    dataset = dataset.shuffle(buffer_size=10 * BATCH_SIZE, reshuffle_each_iteration=True)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)
    # dataset = dataset.repeat()

    return dataset

In [34]:

tfr_dataset = get_dataset(TRAINING_FILENAMES)
print(tfr_dataset)

tfr_testdata = get_dataset(VALID_FILENAMES)
print(tfr_testdata)

<PrefetchDataset shapes: ((None, 400, 400, 3), (None,)), types: (tf.float32, tf.int32)>


In [35]:
def get_cnn():
    model = tf.keras.Sequential([
      
    tf.keras.layers.Conv2D(kernel_size=3, filters=32, padding='same', activation='relu', input_shape=[400, 400, 3]),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    
    # tf.keras.layers.Conv2D(kernel_size=3, filters=64, padding='same', activation='relu'),
    # tf.keras.layers.MaxPooling2D(pool_size=2),
    
    # tf.keras.layers.Conv2D(kernel_size=3, filters=128, padding='same', activation='relu'),
    # tf.keras.layers.MaxPooling2D(pool_size=2),
    
    # tf.keras.layers.Conv2D(kernel_size=3, filters=256, padding='same', activation='relu'),
    # tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(26,'softmax')
    ])

    # opt = SGD(learning_rate=learning_rate)
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(loss= tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer=optimizer, metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
    # model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model

model = get_cnn()
model.summary()

2022-04-17 12:17:48.099347: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 133120000 exceeds 10% of free system memory.
2022-04-17 12:17:48.224693: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 133120000 exceeds 10% of free system memory.
2022-04-17 12:17:48.283850: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 133120000 exceeds 10% of free system memory.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 400, 400, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 200, 200, 32)     0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 1280000)           0         
                                                                 
 dense (Dense)               (None, 26)                33280026  
                                                                 
Total params: 33,280,922
Trainable params: 33,280,922
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(tfr_dataset, 
          # steps_per_epoch=1513/BATCH_SIZE, 
          epochs=20, verbose=1)

Epoch 1/20


/home/nwoke/.local/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)
2022-04-17 12:18:00.904245: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 133120000 exceeds 10% of free system memory.
2022-04-17 12:18:00.936728: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 133120000 exceeds 10% of free system memory.


     12/Unknown - 45s 3s/step - loss: 7360.5508 - sparse_categorical_accuracy: 0.0312

In [ ]:
acc = history.history['sparse_categorical_accuracy']
loss = history.history['loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
# plt.plot(acc, loss, 'r', label='loss against Training accuracy')
plt.title('Training accuracy')
plt.legend(loc=0)
plt.figure()

plt.show()

In [ ]:
# tfr_testdata = get_dataset(VALID_FILENAMES)
validation_array = np.array(list(tfr_testdata.unbatch().as_numpy_iterator()))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


In [ ]:
test_x = np.stack(validation_array[:,0])
test_y = np.stack(validation_array[:,1])
print(len(test_y))
# # Use the model to predict the labels
test_predictions = model.predict(test_x)
test_y_pred = np.argmax(test_predictions, axis=1)
test_y_true = test_y


144


In [ ]:
print(test_y_pred)
print(test_y_true)

[ 4  5  6  5  0 17 25 17  8  2 19  4  9  9  0 25 16 15  2  4 14 14  4 11
  4 20  8 18 25  0  0  6  4 14 18  7  1  2 19  1  2 11 25 13 25 14 11  6
  4 25 20  8 25  2  2 23  4 25  0 22  0 20 22 23 13 25  2  9  5 25 11  0
 19 11  6  8 13  4  9 14 16 24  9  8 11 13  1 11  5 25  5  5  2 24 17  1
  5  6  3  8  0  9  5 24 17  4 17 16  2 14 17 25  1 18 15 13 25  7 17 17
 11  5  7  1  3 15 18 17 24 13  2 16  4  0  2  1  9 14  0  2  5  5 24  5]
[ 7  3  7  3 19 17 25 17 13  3 19  4  9 10  0 20 16 15  6  4 15 14 12 21
 12 20 16 18 10  0  1  7  7 14 18  6  1  1 19 17 15 14  9 19 21  7 11  6
 13 25 20  3 12 13  2 23 12  9  0 22  1 20 22 24 12 22  2 14  5 20 11  8
 19 14  6 16 12 12  9  7 16 24  9 24 11 14  1 19  5 21 18  5 20 24 17  1
  5  1  5 24  0  9  8 24 10  4 10  9 21 14 20 25  1 18 15 13 25  7 17 17
 11  3  7 17  3 21 12 10 24 15  1 15  4  0  2 10  9 15  7  6  5  5 24  5]


In [ ]:
import sklearn 
from sklearn.metrics import accuracy_score
# Evaluating model accuracy and logging it as a scalar for TensorBoard hyperparameter visualization.
accuracy = accuracy_score(test_y_true, test_y_pred)
accuracy
# tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)
# logging.info('Test accuracy:{}'.format(accuracy))

In [ ]:
tfr_testdata = get_dataset(VALID_FILENAMES)
steps_per_epoch = 145/BATCH_SIZE
loss, accuracy = model.evaluate(tfr_testdata
                                # , steps_per_epoch=steps_per_epoch, BATCH_SIZE=BATCH_SIZE
                                )

/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


5/5 [==============================] - 1s 84ms/step - loss: 189.6040 - sparse_categorical_accuracy: 0.4931


In [ ]:
print(loss)
print(accuracy)

189.6040496826172
0.4930555522441864


In [ ]:

steps_per_epoch = 145/BATCH_SIZE
y_pred = model.predict(tfr_testdata, steps_per_epoch, BATCH_SIZE)


In [ ]:

pred_len = []
for i in range(len(y_pred[:10])):

  
  class_label = np.argmax(y_pred[i])
  print(class_label)
  pred_len.append(class_label)
print(len(pred_len))
# pred = np.argmax(y_pred, axis=1) [:]
# test_labels = np.argmax(, axis=1) [:]

# for i in range(20):
#   print("predicted{}--> Expected{}".format(pred[i], test_labels[i]))


13
4
3
14
5
16
22
9
14
14
10


In [ ]:
print(len(y_pred[:]))


144


In [ ]:
y =[]
for (pred,(a,b)) in zip(y_pred,tfr_testdata):
  print("predicted: " , np.argmax(pred), "Actual Label: "+ str(b.numpy()))
  y.append(b.numpy())

predicted:  8 Actual Label: [12  5  6  3 10 21  9  9 10  2  8  9  4 19  7 24  3 10  6 15 16 11 18 24
  0  1 24 15 25 14  9 25]
predicted:  8 Actual Label: [14 14 12 22  9 12 15 21  1 17 16 21 12  2 17  7  7  1 14  6 13  4 17  7
  5 20 19  7  1 14 11  6]
predicted:  8 Actual Label: [ 1  7 15  5  5 19 18  1  7 17  2 22 20 11 24 10  9  3 12 24  5  3  3 17
  9 13  4 14 10  0  8 15]
predicted:  0 Actual Label: [19  7 21 20 24  0 25 16 12 24 23 12 16  5 18  9 24 20 20  5  3 18  1 13
  0 12  6 11  5 25  0  1]
predicted:  9 Actual Label: [ 4 14 10 19 15  7 17 21 22  1 17 19 20 13 15 20]
